In [ ]:
import kagglehub
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf
import os

In [ ]:
path = kagglehub.dataset_download("pythonafroz/solar-panel-images")

100%|██████████| 305M/305M [00:03<00:00, 81.7MB/s]

Extracting files...


In [ ]:
print("Dataset Path:", path)
print("Directory Contents:", os.listdir(path))

Dataset Path: /root/.cache/kagglehub/datasets/pythonafroz/solar-panel-images/versions/2
Directory Contents: ['Faulty_solar_panel']


In [ ]:
# Eğitim ve doğrulama için ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2   # %20 doğrulama verisi ayır
)

In [ ]:
training_set = train_datagen.flow_from_directory(
    '/root/.cache/kagglehub/datasets/pythonafroz/solar-panel-images/versions/2/Faulty_solar_panel',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 711 images belonging to 6 classes.


In [ ]:
validation_set = train_datagen.flow_from_directory(
    '/root/.cache/kagglehub/datasets/pythonafroz/solar-panel-images/versions/2/Faulty_solar_panel',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 174 images belonging to 6 classes.


In [ ]:
# test seti
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_set = test_datagen.flow_from_directory(
    '/root/.cache/kagglehub/datasets/pythonafroz/solar-panel-images/versions/2/Faulty_solar_panel',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 885 images belonging to 6 classes.


In [ ]:
print("Sınıflar:", training_set.class_indices)

Sınıflar: {'Bird-drop': 0, 'Clean': 1, 'Dusty': 2, 'Electrical-damage': 3, 'Physical-Damage': 4, 'Snow-Covered': 5}


In [ ]:
# InceptionV3 modelini yükle
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(6, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=x)

In [ ]:
# Önceden eğitilmiş katmanları dondur
for layer in base_model.layers:
    layer.trainable = False

# Modeli derle
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
for layer in base_model.layers[-30:]:
    layer.trainable = True

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 111, 111, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 111, 111, 32)   │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 111, 111, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 109, 109, 32)   │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 109, 109, 32)   │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 109, 109, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 109, 109, 64)   │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 109, 109, 64)   │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 109, 109, 64)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 54, 54, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 54, 54, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 54, 54, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 54, 54, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 52, 52, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 52, 52, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activat

 Total params: 22,065,830 (84.17 MB)

 Trainable params: 5,419,078 (20.67 MB)

 Non-trainable params: 16,646,752 (63.50 MB)

In [ ]:
# Model eğitimi
history = model.fit(
    training_set,
    epochs=10,
    validation_data=validation_set
)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


23/23 ━━━━━━━━━━━━━━━━━━━━ 173s 7s/step - accuracy: 0.4530 - loss: 1.4575 - val_accuracy: 0.5977 - val_loss: 1.1656
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 152s 6s/step - accuracy: 0.7624 - loss: 0.7700 - val_accuracy: 0.7241 - val_loss: 0.8705
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 202s 6s/step - accuracy: 0.8772 - loss: 0.4933 - val_accuracy: 0.7701 - val_loss: 0.7035
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 198s 6s/step - accuracy: 0.8977 - loss: 0.3709 - val_accuracy: 0.7586 - val_loss: 0.6745
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 150s 6s/step - accuracy: 0.9408 - loss: 0.2862 - val_accuracy: 0.7874 - val_loss: 0.6249
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 203s 6s/step - accuracy: 0.9403 - loss: 0.2200 - val_accuracy: 0.8218 - val_loss: 0.6446
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 149s 6s/step - accuracy: 0.9664 - loss: 0.1801 - val_accuracy: 0.8276 - val_loss: 0.5603
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 145s 6s/step - accuracy: 0.9778 - loss: 0.1289 - val_accuracy: 0.8103 - val_loss: 0.

In [ ]:
test_loss, test_acc = model.evaluate(test_set)
print(f"Test Doğruluğu: {test_acc}")

28/28 ━━━━━━━━━━━━━━━━━━━━ 123s 4s/step - accuracy: 0.9629 - loss: 0.1837
Test Doğruluğu: 0.9672316312789917


In [ ]:
# Test edilecek görselin yolu
test_image_path = "güneş-paneli.jpeg"

test_image = image.load_img(test_image_path, target_size=(224, 224))
test_image_array = image.img_to_array(test_image)
test_image_array = np.expand_dims(test_image_array, axis=0)
test_image_array /= 255.0  # Piksel değerlerini normalleştir

prediction = model.predict(test_image_array)

class_indices = training_set.class_indices
classes = list(class_indices.keys())  # Sınıf isimlerini listele
predicted_class_index = np.argmax(prediction, axis=1)[0]  # En yüksek olasılığa sahip sınıfın indeksini al
predicted_class = classes[predicted_class_index]  # Tahmin edilen sınıfın ismini al

print(f"Güneş panelindeki arıza türü: {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
Güneş panelindeki arıza türü: Snow-Covered
